# RNNs for text generation experiment


## Imports

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import numpy as np
import random
import string
import tensorflow as tf
from six.moves import range
import sys
sys.path.append("/Users/gamal/git_local_repo/playground/utils")
import utils

# Load data

In [2]:
url = 'http://mattmahoney.net/dc/'
filename = utils.maybe_download(url,  "text8.zip", 31344016)

Found and verified text8.zip


In [9]:
text = utils.read_data(filename)
print('number of characters %d' % len(text))

number of characters 100000000


## References
deep learning course at Udacity